In [17]:
import json

import chardet
import pandas as pd

In [18]:
train_dataset = []
dev_dataset = []
test_dataset = []

train_index_label = pd.read_csv("dataset/train.txt")
test_index_label = pd.read_csv("dataset/test_without_label.txt").values
train_count = dict(train_index_label["tag"].value_counts() * 0.8 // 1)

train_neg = train_index_label.loc[train_index_label["tag"] == "negative"].sample(n=int(train_count["negative"])).values
train_pos = train_index_label.loc[train_index_label["tag"] == "positive"].sample(n=int(train_count["positive"])).values
train_neu = train_index_label.loc[train_index_label["tag"] == "neutral"].sample(n=int(train_count["neutral"])).values
dev_neg = train_index_label[~train_index_label["guid"].isin(train_neg[:, 0])].loc[
    train_index_label["tag"] == "negative"].values
dev_pos = train_index_label[~train_index_label["guid"].isin(train_pos[:, 0])].loc[
    train_index_label["tag"] == "positive"].values
dev_neu = train_index_label[~train_index_label["guid"].isin(train_neu[:, 0])].loc[
    train_index_label["tag"] == "neutral"].values

In [19]:
def add2dataset(data, dataset):
    for i in range(data.shape[0]):
        guid = str(int(data[i, 0]))
        with open("dataset/data/" + guid + ".txt", "rb") as f:
            encoding = chardet.detect(f.read())["encoding"]
        if encoding == "GB2312":
            encoding = "GBK"

        with open("dataset/data/" + guid + ".txt", encoding=encoding) as f:
            dataset.append({
                "text": f.read().rstrip("\n"),
                "label": (data[i, 1] if data[0, 1] != "NaN" else None),
                "img": "dataset/data/" + guid + ".jpg"
            })
    return dataset


train_dataset = add2dataset(train_neg, train_dataset)
train_dataset = add2dataset(train_pos, train_dataset)
train_dataset = add2dataset(train_neu, train_dataset)

dev_dataset = add2dataset(dev_neg, dev_dataset)
dev_dataset = add2dataset(dev_pos, dev_dataset)
dev_dataset = add2dataset(dev_neu, dev_dataset)

test_dataset = add2dataset(test_index_label, test_dataset)

with open("dataset/train.json", "w", encoding="utf-8") as f:
    json.dump(train_dataset, f)

with open("dataset/dev.json", "w", encoding="utf-8") as f:
    json.dump(dev_dataset, f)

with open("dataset/test.json", "w", encoding="utf-8") as f:
    json.dump(test_dataset, f)